In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

RTS_DATA = "/mnt/d/rts/"
METADATA = RTS_DATA + 'metadata'
VIDEOS = RTS_DATA + '0'
REMOTE_VIDEOS = "/mnt/rts"
OUTDIR = 'data'



/home/kikohs/work/rts


In [12]:
from datetime import datetime
import meilisearch
import orjson
import pandas as pd
import zipfile
import os
import io
from typing import Dict, List, Optional, Tuple

In [6]:
# LOCAL imports
import rts
from rts.ingest import read_video_folder_index

LOG = rts.utils.get_logger()

In [5]:
vidx = read_video_folder_index(os.path.join(OUTDIR, 'vidx.json'))

In [3]:
client = meilisearch.Client('http://localhost:7700', '1234')
# client.get_keys()
 
# int(datetime.now().timestamp())
# client.create_index('books', {'primaryKey': 'reference_number'})

NameError: name 'meilisearch' is not defined

In [21]:
def parse_item_sequences(data: Dict, media: Dict) -> Dict:
    seqs = {}
    for id in data.get('idSequence', []):
        seq = {
            'guid': media['guid'],
            'mediaId': media['mediaId'],
            'seqid': id,
            'geo': [],
            'mat': [],
            'pp': []
        }
        seqs[seq['seqid']] = seq

    geo = data.get('VisuelsGEOSequence', [])
    for g in geo:
        seqid, loc = g.split('@')
        seqs[seqid]['geo'].append(loc.lower())

    mat = data.get('VisuelsMATSequence', [])
    for m in mat:
        seqid, mat = m.split('@')
        seqs[seqid]['mat'].append(mat.lower())
    
    pp = data.get('VisuelsPPSequence', [])
    for p in pp:
        seqid, name = p.split('@')
        seqs[seqid]['pp'].append(name.lower())
    
    return seqs


def parse_item(raw: Dict, vidx: Dict) -> Optional[Dict]:
    def find_media_id(support: Dict) -> Tuple[Optional[int], Optional[str]]:
        for i, d in enumerate(support):
            if d.startswith('Z'):
                return i, d
        return None, None

    d = raw['response']['docs'][0]
    try:
        seq_idx, media_id = find_media_id(d['idSupport'])
    except KeyError as e:
        LOG.error(e)
        return None

    duration = d['DureeMediaSec'][seq_idx]
    ratio = d['RatioMedia'][seq_idx]

    media = {
        'guid': d['Guid'],
        'mediaId': media_id,
        'mediaFolderPath': vidx.get(media_id),
        'mediaDuration': duration,
        'ratio': ratio,
        'formatType': d['FormatMedia'][seq_idx],
        'formatResolution': d['DefinitionMedia'][seq_idx],
        'materialType': d['TypeMaterielMedia'][seq_idx],
        'publishedDate': d.get('DatePublication'),
        'publishedBy': d.get('CanalPublication'),

        'rights': d.get('SemaphoreDroit'),
        'categoryName': d.get('CategorieAsset'),
        'assetType': d.get('TypeAsset'),
        'contentType': d.get('TypeContenu'),
        'backgoundType': d.get('idTypeBackground'),
        'collection': d.get('Collection'),
        
        'title': d.get('Titre'),
        'resume': d.get('Resume'),
        'geoTheme': d.get('ThematiquesGEO'),
        'resumeSequence': d.get('ResumeSequence'),
        'created': int(datetime.now().timestamp()),
        'published': int(datetime.strptime(d.get('DatePublication'), "%Y-%m-%dT%H:%M:%S%z").timestamp())
    }

    seqs = parse_item_sequences(d, media)
    media['sequences'] = seqs

    return media


def read_txt_transcript(srtz: zipfile.Path, media: Dict) -> Optional[Dict]:
    try:
        ts = {
            'guid': media['guid'],
            'mediaId': media['mediaId'],
            'version': 1,
            'text': ''
        }
        path = media['guid'] + '_0.txt'
        current = srtz.joinpath(path)
        with current.open('r') as fp:
            wrap = io.TextIOWrapper(fp, encoding='latin-1')
            ts['text'] = wrap.read()
    except Exception as e:
        LOG.error('Read txt ts:', e)
        return None
    return ts


def read_metadata_zippart(root_dir: str, part_num: int, vidx: Dict) -> Dict:
    path = os.path.join(root_dir, f'{part_num}.zip')
    medias = {}
    with zipfile.ZipFile(path, 'r') as z:
        jsons = zipfile.Path(z, f'{part_num}/jsonData/')
        for j in jsons.iterdir():
            with j.open('r') as fp:
                js = orjson.loads(fp.read())
                media = parse_item(js, vidx)
                if media:
                # ts = read_txt_transcript(srts, media)
                # media['ts'] = ts
                    medias[media['guid']] = media

    return medias


def read_all_metadata(root_dir: str, vidx: Dict) -> Dict:
    all_d = None
    for i in range(0, 10):
        d = read_metadata_zippart(root_dir, i, vidx)
        all_d.update(d)
    
    return all_d


medias = read_metadata_zippart(METADATA, 0, vidx)

ERROR:RTS:'idSupport'
{'CategorieAsset': 'Programme', 'SemaphoreDroit': 'Inconnu', 'DureeSec': 3821, 'DateModification': '2018-04-17T01:40:48Z', 'HeurePublication': '09:39:00', 'idCollection': '408', 'ApplicationModification': 'TMS', 'NumeroEpisode': 101, 'idTypeProduction': '6', 'Collection': 'Ski alpin', 'idTypeAsset': '5', 'idStatutArchivage': '3', 'idCategorieAsset': 'PROGR', 'aVignette': False, 'DatePublication': '2011-10-22T09:39:00Z', 'DateCreation': '2011-09-29T12:12:26Z', 'Guid': 'AA1109014960', 'TypeProduction': 'Transmission', 'key': 'asset-14961', 'StatutArchivage': 'Terminé', 'Titre': 'Géant dames, 1re manche, Sölden', 'environment': 'production', 'ApplicationCreation': 'STRADA', 'batchid': 1645384501, 'role': 'asset', 'TypeAsset': 'Retransmission sportive', 'Duree': '01:03:41:00', 'idTypeContenu': ['Multiple'], 'idCanalPublication': ['588'], 'id': '14961', 'UtilisateurCreation': 'STRADA', 'Saison': '2011', 'CodeCollection': 'SKIAL', 'TypeContenu': ['Valeurs multiples'], '

KeyError: 

In [3]:
import whisper

model = whisper.load_model("base")
# result = model.transcribe("audio.mp3")
# print(result["text"])

In [12]:

vidx = get_video_folder_index(REMOTE_VIDEOS)

In [25]:


# vidx_tojson(vidx, '/home/kikohs/work/rts/data/vidx.json')

In [19]:
%pwd

'/home/kikohs'

In [20]:
os.path.join(OUTDIR, 'vidx.json')

'./data/vidx.json'

In [26]:
vid2 = read_vidx('/home/kikohs/work/rts/data/vidx.json')